In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import traceback

In [6]:
api_key = ""

In [7]:
def get_summoners_id_by_rank(region, rank):
    api_url = "https://"+region+".api.riotgames.com/lol/league/v4/"+rank+"/by-queue/RANKED_SOLO_5x5?api_key="+api_key
    resp = requests.get(api_url)
    data = resp.json()
    return data


def get_player_info_by_summoner_id(region, summoner_id):
    api_url = "https://"+region+".api.riotgames.com/lol/summoner/v4/summoners/"+summoner_id+"?api_key="+api_key
    resp = requests.get(api_url)
    if resp.status_code == 200:
        data = resp.json()
        return data
    else:
        return False


def get_info_by_puuid(continent, puuid):
    api_url = "https://"+continent+".api.riotgames.com/riot/account/v1/accounts/by-puuid/"+puuid+"?api_key="+api_key
    resp = requests.get(api_url)
    if resp.status_code == 200:
        data = resp.json()
        return data
    else:
        return False


def get_player_info_by_puuid(region, continent, summoners):
    count = 0
    data = {"game_name": [], "tag_line": [], "puuid": []}
    
    for summoner in summoners:
        if count % 999 == 0 and count != 0:
            print("1 minute break")
            time.sleep(60)
            
        if count % 100 == 0:
            print(f"Count: {count} -", datetime.now())
        
        try:
            player_ids = get_player_info_by_summoner_id(region, summoner["summonerId"])        
            player_info = get_info_by_puuid(continent, player_ids["puuid"])
            
            if player_info != False: 
                data["game_name"].append(player_info["gameName"])
                data["tag_line"].append(player_info["tagLine"])
                data["puuid"].append(player_ids["puuid"])
        except Exception:
            print(count, summoner["summonerId"])
            traceback.print_exc()
        
        count += 1

    df = pd.DataFrame(data)
    df.to_csv("data.csv")

In [8]:
regions = ["br1", "eun1", "euw1", "jp1", "kr", "la1", "la2", "na1", "oc1", "ph2", "ru", "sg2", "th2", "tr1", "tw2", 'vn2']
ranks = ["challengerleagues", "grandmasterleagues", "masterleagues"]
continents = ["americas", "asia", "esports", "europe"]

region = regions[0]
rank = ranks[0]
continent = continents[0]

summoners = get_summoners_id_by_rank(region, rank)
get_player_info_by_puuid(region, continent, summoners["entries"])

Count: 0 -  2024-05-28 11:58:33.231195
Count: 100 -  2024-05-28 12:01:02.557325
